# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
from __future__ import division
import graphlab
import math
import string

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.3.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Data preperation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] GraphLab Create v1.8.3 started. Logging: C:\Users\NikolaiA\AppData\Local\Temp\graphlab_server_1459892727.log.0


Now, let us see a preview of what the dataset looks like.

In [3]:
print len(products)
products


183531


name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

In [5]:
print products[200]

{'rating': 5.0, 'review': 'This is exactly like the one I had when I was a kid. My son really enjoys it and it keeps him entertained.', 'name': 'My Quiet Book, Fabric Activity Book for Children'}


Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [6]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

review_without_puctuation = products['review'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_puctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [7]:
products[200]['word_count']

{'a': 1L,
 'and': 1L,
 'enjoys': 1L,
 'entertained': 1L,
 'exactly': 1L,
 'had': 1L,
 'him': 1L,
 'i': 2L,
 'is': 1L,
 'it': 2L,
 'keeps': 1L,
 'kid': 1L,
 'like': 1L,
 'my': 1L,
 'one': 1L,
 'really': 1L,
 'son': 1L,
 'the': 1L,
 'this': 1L,
 'was': 1L,
 'when': 1L}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [8]:
print len(products)
products = products[products['rating'] != 3]
print len(products)

183531
166752


Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [9]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 3L, 'highly': 1L, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3L, 'ingenious':1L, 'love': 2L, 'is': ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'all': 2L,'help': 1L, 'cried': 1L, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'habit': 1L, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'bound': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2L, 'all': 1L,'right': 1L, 'able': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'fantastic':1L, 'help': 1L, 'give': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'all': 1L, 'standarad':1L, 'another': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2L, 'nannys': 1L,'just': 1L, 'sleep': 2L, ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [10]:
train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [11]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

In [12]:
sentiment_model

Class                         : LogisticClassifier

Schema
------
Number of coefficients        : 121713
Number of examples            : 133416
Number of classes             : 2
Number of feature columns     : 1
Number of unpacked features   : 121712

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : auto
Solver iterations             : 6
Solver status                 : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)           : 6.9744

Settings
--------
Log-likelihood                : inf

Highest Positive Coefficients
-----------------------------
word_count[mobileupdate]      : 41.9847
word_count[placeid]           : 41.7354
word_count[labelbox]          : 41.151
word_count[httpwwwamazoncomreviewrhgg6qp7tdnhbrefcmcrprcmtieutf8asinb00318cla0nodeid]: 40.0454
word_count[knobskeeping]      : 36.2091

Lowest Negative Coefficients
------------

**Aside**. You may get an warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [13]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

In [14]:
print len (weights)
print weights[weights['name']=='word_count']


121713
+------------+--------------+-------+------------------+--------+
|    name    |    index     | class |      value       | stderr |
+------------+--------------+-------+------------------+--------+
| word_count |  recommend   |   1   |  0.303815600015  |  None  |
| word_count |    highly    |   1   |  1.49183015276   |  None  |
| word_count | disappointed |   1   |  -3.95748618393  |  None  |
| word_count |     love     |   1   |  1.43301685439   |  None  |
| word_count |      it      |   1   | 0.00986646490307 |  None  |
| word_count |    planet    |   1   | -0.797764553926  |  None  |
| word_count |     and      |   1   |  0.048449573172  |  None  |
| word_count |     bags     |   1   |  0.165541436615  |  None  |
| word_count |    wipes     |   1   | -0.0949937947269 |  None  |
| word_count |     not      |   1   | -0.817151716331  |  None  |
+------------+--------------+-------+------------------+--------+
[? rows x 5 columns]
Note: Only the head of the SFrame is printed. Th

In [15]:
print weights['name'].unique()

['word_count', '(intercept)']


There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [16]:
num_positive_weights = len(weights[weights['value']>=0])
num_negative_weights = len(weights[weights['value']<0])

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 68419 
Number of negative weights: 53294 


**Quiz question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [17]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'and': 2L, 'all': 1L,'love': 1L, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'and': 1L, 'wall': 1L,'them': 1L, 'decals': ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'all': 1L, 'money': 1L,'into': 1L, 'it': 3L, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [18]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

In [19]:
dic=sample_test_data[0]['word_count']
print dic["all"] 
sample_test_data['word_count'][0]

1


{'absolutely': 1L,
 'again': 1L,
 'all': 1L,
 'and': 2L,
 'baby': 1L,
 'book': 1L,
 'born': 1L,
 'boy': 1L,
 'daughterinlaw': 1L,
 'for': 1L,
 'grandson': 1L,
 'he': 1L,
 'i': 1L,
 'in': 1L,
 'it': 2L,
 'love': 1L,
 'my': 2L,
 'of': 1L,
 'purchased': 1L,
 'receive': 1L,
 'same': 1L,
 'scripture': 1L,
 'the': 3L,
 'thrilled': 1L,
 'to': 1L,
 'version': 1L,
 'was': 2L,
 'when': 1L}

In [20]:
def calc_score(mod,tst):
    res=mod[mod['name']=='(intercept)'][0]['value']
    for d in mod[mod['name']=='word_count']:
        if d['index'] in tst:
            res+=d['value']*tst[d['index']]
    return res

In [53]:
print weights[weights['name']=='(intercept)']['value']

[1.3033708054360078, ... ]


In [68]:
r=calc_score(weights,sample_test_data['word_count'][0])
print r

6.73461972706


In [69]:
r=calc_score(weights,sample_test_data['word_count'][1])
print r

-5.73413099676


That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [70]:
r=calc_score(weights,sample_test_data['word_count'][2])
print r

-14.6684604045


In [22]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [23]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print scores

[6.734619727059016, -5.73413099676019, -14.668460404468401]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [65]:
def calc_y(mod,tst): 
    if calc_score(mod,tst)>0:
        return 1
    else:
        return 0-1


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [25]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data,output_type="margin")

Class predictions according to GraphLab Create:
[6.734619727059016, -5.73413099676019, -14.668460404468401]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [71]:
print calc_score(weights,sample_test_data['word_count'][0])
print calc_y(weights,sample_test_data['word_count'][0])
print calc_score(weights,sample_test_data['word_count'][1])
print calc_y(weights,sample_test_data['word_count'][1])
print calc_score(weights,sample_test_data['word_count'][2])
print calc_y(weights,sample_test_data['word_count'][2])

6.73461972706
1
-5.73413099676
-1
-14.6684604045
-1


In [75]:
def calc_pr(mod,tst):
    return 1/(1+exp(-calc_score(mod,tst)))

In [73]:
from math import exp

In [76]:
print calc_score(weights,sample_test_data['word_count'][0])
print calc_pr(weights,sample_test_data['word_count'][0])

6.73461972706
0.998812384838


In [78]:
print calc_pr(weights,sample_test_data['word_count'][1])
print calc_pr(weights,sample_test_data['word_count'][2])

0.0032232681818
4.26155799666e-07


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [26]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data, output_type='probability')

Class predictions according to GraphLab Create:
[0.9988123848377194, 0.0032232681818008973, 4.2615579966559177e-07]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

In [ ]:
# second 0.0032232681818

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [82]:
p=sentiment_model.predict(sample_test_data, output_type='probability')

In [85]:
x=sample_test_data
x['pr']=sentiment_model.predict(sample_test_data, output_type='probability')

In [88]:
x.topk('pr',k=2,reverse=False)

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	pr	float

Rows: 2

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|   Our Baby Girl Memory Book   | Absolutely love it and all... |  5.0   |
| Wall Decor Removable Decal... | Would not purchase again o... |  2.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+-----------------+
|           word_count          | sentiment |        pr       |
+-------------------------------+-----------+-----------------+
| {'and': 2L, 'all': 1L, 'lo... |     1     |  0.998812384838 |
| {'and': 1L, 'wall': 1L, 't... |     -1    | 0.0032232681818 |
+-------------------------------+-----------+-----------------+
[2 rows x 6 columns]

In [92]:
top_rev=test_data
top_rev['pr']=sentiment_model.predict(test_data, output_type='probability')
top_rev=top_rev.topk('pr',k=20,reverse=False)
top_rev.print_rows(num_rows=20)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Britax Decathlon Convertib... | I researched a few differe... |  4.0   |
| Ameda Purely Yours Breast ... | As with many new moms, I c... |  4.0   |
| Traveling Toddler Car Seat... | I am sure this product wor... |  2.0   |
| Shermag Glider Rocker Comb... | After searching in stores ... |  4.0   |
| Cloud b Sound Machine Soot... | First off, I love plush sh... |  5.0   |
| JP Lizzy Chocolate Ice Cla... | I got this bag as a presen... |  4.0   |
| Fisher-Price Rainforest Me... | My daughter wasn't able to... |  5.0   |
| Lilly Gold Sit 'n' Stroll ... | I just completed a two-mon... |  5.0   |
|  Fisher-Price Deluxe Jumperoo | I had already decided that... |  5.0   |
| North States Supergate Pre... | I got a couple of these to... |  4.0   |
|   Munchkin Mozart Magic

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this excercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [93]:
bot_rev=test_data
bot_rev['pr']=sentiment_model.predict(test_data, output_type='probability')
bot_rev=bot_rev.topk('pr',k=20,reverse=True)
bot_rev.print_rows(num_rows=20)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Jolly Jumper Arctic Sneak ... | I am a "research-aholic" i... |  5.0   |
| Levana Safe N'See Digital ... | This is the first review I... |  1.0   |
| Snuza Portable Baby Moveme... | I would have given the pro... |  1.0   |
| Fisher-Price Ocean Wonders... | We have not had ANY luck w... |  2.0   |
| VTech Communications Safe ... | This is my second video mo... |  1.0   |
| Safety 1st High-Def Digita... | We bought this baby monito... |  1.0   |
| Chicco Cortina KeyFit 30 T... | My wife and I have used th... |  1.0   |
| Prince Lionheart Warmies W... | *****IMPORTANT UPDATE*****... |  1.0   |
| Valco Baby Tri-mode Twin S... | I give one star to the dim... |  1.0   |
| Adiri BPA Free Natural Nur... | I will try to write an obj... |  2.0   |
| Munchkin Nursery Projec

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [96]:
#def get_classification_accuracy(model, data, true_labels):
def get_classification_accuracy(model, data):
    # First get the predictions
    ## YOUR CODE HERE
    d=data
    d['pr']=model.predict(d, output_type='probability')
    d['csentiment']=d['pr'].apply(lambda rating : +1 if rating > 0.5 else -1)
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    corr=len(d[d['sentiment']==d['csentiment']])

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy=corr/len(d)
    
    return accuracy

In [98]:
get_classification_accuracy(sentiment_model, test_data)

0.9145368370530358

In [99]:
get_classification_accuracy(sentiment_model, train_data)

0.979440247046831

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [ ]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

In [ ]:
# 0.91
# No

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [100]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [101]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [102]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [103]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [104]:
print train_data[0]['word_count']

{'and': 3L, 'love': 1L, 'it': 3L, 'highly': 1L, 'osocozy': 1L, 'bags': 1L, 'leak': 1L, 'moist': 1L, 'does': 1L, 'recommend': 1L, 'was': 1L, 'wipes': 1L, 'disappointed': 1L, 'early': 1L, 'not': 2L, 'now': 1L, 'holder': 1L, 'wipe': 1L, 'keps': 1L, 'wise': 1L, 'i': 1L, 'planet': 1L, 'my': 2L, 'came': 1L}


Since we are only working with a subet of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [105]:
print train_data[0]['word_count_subset']

{'love': 1L, 'disappointed': 1L}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [106]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133416
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 20
PROGRESS: Number of coefficients    : 21
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+
PROGRESS: | 1         | 2        | 0.213013     | 0.862917          |
PROGRESS: | 2         | 3        | 0.369022     | 0.865713          |
PROGRESS: | 3         | 4        | 0.486028     | 0.866478          |
PROGRESS: | 4         | 5        | 0.603035     | 0.866748          |
PROGRESS: | 5         | 6        | 0.720042     | 0.866815          |
PROGRESS: | 6         | 7  

Class                         : LogisticClassifier

Schema
------
Number of coefficients        : 21
Number of examples            : 133416
Number of classes             : 2
Number of feature columns     : 1
Number of unpacked features   : 20

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : auto
Solver iterations             : 6
Solver status                 : SUCCESS: Optimal solution found.
Training time (sec)           : 0.863

Settings
--------
Log-likelihood                : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]      : 1.6773
word_count_subset[perfect]    : 1.5145
word_count_subset[love]       : 1.3654
(intercept)                   : 1.2995
word_count_subset[easy]       : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed]: -2.3551
word_count_subset[return

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [107]:
#get_classification_accuracy(simple_model, test_data, test_data['sentiment'])
get_classification_accuracy(simple_model, test_data)

0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [108]:
simple_model.coefficients

name,index,class,value
(intercept),None,1,1.2995449552
word_count_subset,disappointed,1,-2.35509250061
word_count_subset,love,1,1.36543549368
word_count_subset,well,1,0.504256746398
word_count_subset,product,1,-0.320555492996
word_count_subset,loves,1,1.67727145556
word_count_subset,little,1,0.520628636025
word_count_subset,work,1,-0.621700012425
word_count_subset,easy,1,1.19366189833
word_count_subset,great,1,0.94469126948


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [109]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+-----------------+
|        name       |    index     | class |      value      |
+-------------------+--------------+-------+-----------------+
| word_count_subset |    loves     |   1   |  1.67727145556  |
| word_count_subset |   perfect    |   1   |  1.51448626703  |
| word_count_subset |     love     |   1   |  1.36543549368  |
|    (intercept)    |     None     |   1   |   1.2995449552  |
| word_count_subset |     easy     |   1   |  1.19366189833  |
| word_count_subset |    great     |   1   |  0.94469126948  |
| word_count_subset |    little    |   1   |  0.520628636025 |
| word_count_subset |     well     |   1   |  0.504256746398 |
| word_count_subset |     able     |   1   |  0.191438302295 |
| word_count_subset |     old      |   1   | 0.0853961886678 |
| word_count_subset |     car      |   1   |  0.058834990068 |
| word_count_subset |     less     |   1   | -0.209709815216 |
| word_count_subset |   product    |   1   | -0.3205554

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [110]:
11

11

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [124]:
weights[weights['index']=='product']['value']
'yes'

'yes'

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [125]:
get_classification_accuracy(sentiment_model, train_data)

0.979440247046831

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [126]:
get_classification_accuracy(simple_model, train_data)

0.8668150746537147

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this excercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [127]:
get_classification_accuracy(sentiment_model, test_data)

0.9145368370530358

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [128]:
get_classification_accuracy(simple_model, test_data)

0.8693004559635229

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [129]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


In [132]:
mc=num_positive/len(train_data)
mc

0.8407087605684476

In [135]:
num_positive2  = (test_data['sentiment'] == +1).sum()
num_negative2 = (test_data['sentiment'] == -1).sum()
print num_positive2
print num_negative2
print num_positive2/len(test_data)

28095
5241
0.842782577394


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [136]:
# 0.84

In [137]:
weights

name,index,class,value
(intercept),None,1,1.30337080544
word_count,recommend,1,0.303815600015
word_count,highly,1,1.49183015276
word_count,disappointed,1,-3.95748618393
word_count,love,1,1.43301685439
word_count,it,1,0.00986646490307
word_count,planet,1,-0.797764553926
word_count,and,1,0.048449573172
word_count,bags,1,0.165541436615
word_count,wipes,1,-0.0949937947268


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?

In [147]:
mydic={}

In [145]:
l=weights[0]
print l

{'index': None, 'name': '(intercept)', 'value': 1.3033708054360078, 'class': 1L}


In [144]:
print l['index']

None


In [148]:
#mydic['first']='second'
print mydic

{}


In [162]:
import nltk
from nltk.corpus import stopwords
stws=stopwords.words('english')

In [163]:
for l in weights:
    if l['index'] not in stws:
        mydic[l['index']]=l['value']
print len(mydic)

121713


In [151]:
len(mydic)

121713

In [155]:
print mydic['the']

-0.00410088996429


In [156]:
import nltk

In [157]:
from nltk.corpus import stopwords

In [160]:
stws=stopwords.words('english')

In [161]:
len(stws)

127

In [ ]:
mydic1=[]